 Import Required Libraries

In [1]:
import ee
import geemap
import pandas as pd
from IPython.display import Image

Authenticate Google Earth Engine

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AZEOvhW0mPL8_XtoQDj9tM-Rs9Mj5pT6BFDkLN9e-M7wBsNI7mCmtA2FMpg

Successfully saved authorization token.


Import Data

In [3]:
#Alaska boundary
alaska_bound = ee.FeatureCollection('TIGER/2018/States').filter(ee.Filter.eq('STUSPS','AK'))

#create section of Ykdelta
lon = 62.83
lat = -153
poi = ee.Geometry.Point(lon,lat)
poi_reprojected = poi.transform('EPSG:4269')
box = [-165.3,62.2,-162.5,63.4]
roi =  ee.Geometry.Rectangle(box)

In [4]:
#Import Digital Elevation Model
Dataset = ee.Image('USGS/3DEP/10m')
Elevation = Dataset.select('elevation')

#change alaska boundary to proper coordinate system
alaska_bound_reprojected = alaska_bound.geometry().transform('EPSG:4269')

#clip elevation to just alaska
Elevation_Alaska = Elevation.clip(alaska_bound_reprojected)

url = Elevation_Alaska.getThumbUrl({
    'min': 0, 'max': 2500 ,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})

# Display
Image(url=url)

In [5]:
Elevation_Alaska_reprojected = Elevation_Alaska.reproject(crs='WGS84')

#get download link
link = Elevation_Alaska.getDownloadURL({
    'scale' : 70,
    'crs' : 'EPSG:4269',
    'fileFormat': 'GeoTIFF',
    'region': roi,
    'name': 'elevation'
})
print(link)

url = Elevation_Alaska.getThumbUrl({
    'min': 0, 'max': 2500 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/9c9a4c1bb69f4e545decb8e385333006-873466afc9a1a25067898180cc770e5d:getPixels


Get Slope from DEM

In [6]:
Slope_Alaska = ee.Terrain.slope(Elevation_Alaska)


url = Slope_Alaska.getThumbUrl({
    'min': 0, 'max': 90 ,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})

# Display
Image(url=url)

In [7]:
#get download link

link = Slope_Alaska.getDownloadURL({
    'scale' : 70,
    'crs' : 'EPSG:4269',
    'fileFormat': 'GeoTIFF',
    'region': roi,
    'name': 'slope'
})
print(link)

url = Slope_Alaska.getThumbUrl({
    'min': 0, 'max': 90 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/46e55a7002689b95176f25dddb543d45-31cadbadb0ee35057ce0e54f54dd93e8:getPixels


In [8]:
Aspect_Alaska = ee.Terrain.aspect(Elevation_Alaska)

url = Aspect_Alaska.getThumbUrl({
    'min': 0, 'max': 360 ,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})

# Display
Image(url=url)

In [9]:
#get download link

link = Aspect_Alaska.getDownloadURL({
    'scale' : 70,
    'crs' : 'EPSG:4269',
    'fileFormat': 'GeoTIFF',
    'region': roi,
    'name': 'aspect'
})
print(link)

url = Aspect_Alaska.getThumbUrl({
    'min': 0, 'max': 360 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/5b97ffd766d4204ac066d1b2652aa056-4dc89c9d9f3a71e1186039a74b10e6fa:getPixels


In [6]:
#NDVI
Dataset = ee.ImageCollection('MODIS/061/MYD13Q1').filterDate('2018-01-01','2018-02-01')

NDVI = Dataset.select('NDVI')
#NDVI_Alaska = NDVI.clip('alaska_bound_reprojected')
NDVI_Alaska = NDVI.mosaic().clip(alaska_bound)
url = NDVI_Alaska.getThumbUrl({
   'min': -2000, 'max': 10000 ,'region':alaska_bound.geometry(), 'dimensions':2048,
   'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

In [11]:
for i in range(2000,2022,1):
    Start_Date = str(i) + '-01-01'
    End_Date = str(i) + '-12-31'

    Dataset = ee.ImageCollection('MODIS/061/MYD13Q1').filterDate(Start_Date,End_Date).select('NDVI')
    NDVI_Alaska = NDVI.mosaic().clip(alaska_bound)
    #get download link
    link = NDVI_Alaska.getDownloadURL({
        'scale' : 50,
        'crs' : 'EPSG:4269',
        'fileFormat': 'GeoTIFF',
        'region': roi,
        'name': 'NDVI_' + str(i)     
    })
    print(Start_Date +": " + link)

url = NDVI_Alaska.getThumbUrl({
    'min': -2000, 'max': 10000 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)
    

2000-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/bede17027561807b4ec9010dafe5a39c-0eaaaf72eef7220105088d60544fc922:getPixels
2001-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/d3c6618534804760994259175c52e0d7-59b8fa74f80a6bdf5b6d2a5c2b9fa546:getPixels
2002-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/1fd47225173bb48b762d09854dd089ef-2442450beacf670846dffbf42ea42c4e:getPixels
2003-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/d9f8402c1529b9f9f306c961ec9b6b62-83a9f3c427b11e6f84c084385e09ad86:getPixels
2004-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/a50f842d8e6b5190892847e823924034-34f70868dd97bdbc50fd90005c114370:getPixels
2005-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/e47847b9ccb0439b2be8c4060f1380fb-a818b9915a8e1f79f492057

Temperature

In [31]:
for i in range(2000,2022,1):
    Start_Date = str(i) + '-01-01'
    End_Date = str(i) + '-12-31'
    Dataset = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').filterDate(Start_Date,End_Date).select('tmmx')
    avg_maxTemp = Dataset.mean().clip(alaska_bound)
    link = avg_maxTemp.getDownloadURL({
        'scale' : 100,
        'crs' : 'EPSG:4269',
        'fileFormat': 'GeoTIFF',
        'region': roi,
        'name': 'avgmaxTemp_' + str(i)     
    })
    print(Start_Date +": " + link)
url = avg_maxTemp.getThumbUrl({
    'min': -670, 'max': 576 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

2000-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/7b2c4c8313f79833f768aec5084ff119-acca58eb9d081b7ba3662c95d558b1a7:getPixels
2001-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/baf7fcd56be094b46b79539ec25275b2-57effa7235726b0b7800f22e92ed26ef:getPixels
2002-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/fbbb8046fef243712827a4ec05274aec-1427342c0718d4dbc04301c89efbbbfd:getPixels
2003-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/8c1a2b1cf1170b241fc7f346e02f6cec-76d30820b49bc68f98de4578254a6b77:getPixels
2004-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/f10d1b6324d60ecd3a4d8d933c4b4895-e327ee9df4a611a0d971b2b72cba3b58:getPixels
2005-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/bef34faac663a08042914aadb6c26bcc-fb911194e36575af5dd398d

Soil Moisture

In [32]:
for i in range(2000,2022,1):
    Start_Date = str(i) + '-01-01'
    End_Date = str(i) + '-12-31'
    Dataset = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').filterDate(Start_Date,End_Date).select('soil')
    soil = Dataset.mean().clip(alaska_bound)
    link = soil.getDownloadURL({
        'scale' : 90,
        'crs' : 'EPSG:4269',
        'fileFormat': 'GeoTIFF',
        'region': roi,
        'name': 'soil_' + str(i)     
    })
    print(Start_Date +": " + link)
url = soil.getThumbUrl({
    'min': -0, 'max': 9000 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

2000-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/10a6e3e8d9f48eb5f01d1bcf09a87b45-6010d6640d52b44b2e7a97bbccc3e8f6:getPixels
2001-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/0ee157f2600d80b0aeb56fd17029a4f5-d89b211757672224d48f27966825c13e:getPixels
2002-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/caf15d393033df09acf0e641acbc42ab-85bbe178be849a29c16729cd78c2cf18:getPixels
2003-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/6a7da5c511e184d820a381c2a7ba348b-3a2f1f9152271a3f3722ef7cfce9ae63:getPixels
2004-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/4fd422aec43a1754eb1eea41db39c7bd-733c9dc530c92942d65fd68b4d76034a:getPixels
2005-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/412059e7492c5f300837236be1643436-05693ca6997144a1de97e92

# Precipitation

In [33]:
for i in range(2000,2022,1):
    Start_Date = str(i) + '-01-01'
    End_Date = str(i) + '-12-31'
    Dataset = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').filterDate(Start_Date,End_Date).select('pr')
    precip = Dataset.sum().clip(alaska_bound)
    link = precip.getDownloadURL({
        'scale' : 90,
        'crs' : 'EPSG:4269',
        'fileFormat': 'GeoTIFF',
        'region': roi,
        'name': 'precip_' + str(i)     
    })
    print(Start_Date +": " + link)
url = precip.getThumbUrl({
    'min': -0, 'max': 9000 ,'region':roi,'dimensions':512,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']})
Image(url=url)

2000-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/2b8a2e4470d3b7b4f7f6870e09053e73-19aabee746559b0855ed654b8c337fff:getPixels
2001-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/257f7ad1288a87b519b65124af75afdf-ce4e74f0dc33af7b6c4c6f9d4b3113f1:getPixels
2002-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/7696898bbfa2e114d4714e92d9e2cb64-79b026a78f5f55a3f20fc1d5507770de:getPixels
2003-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/62a5c40376ef29961f6f86c0fb56c022-66b1bcbfb569129184e76673a5e79f47:getPixels
2004-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/ad754be77475125ae9578f52ab6f5e4d-8d22052dee802a5ec2a66a4540f7d63b:getPixels
2005-01-01: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/3d6921d2c0ae34ef0c28bd8fec6de100-143736c2137ab527c948421